In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
notebook_dir="/content/drive/MyDrive/Colab Notebooks/Transformer"

In [1]:
cd "/content/drive/MyDrive/Colab Notebooks/Transformer"

/content/drive/.shortcut-targets-by-id/17vBJAZeuDCte4t5jYXL1emXc1Rm4jOqV/Transformer


In [4]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")
import slacknotice # オリジナルモジュール slacknotice.send("")

# ライブラリ読み込み

In [5]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [4]:
import numpy as np
import os
import time
import MeCab

import preprocess_utils
import model
import weight_utils

import tensorflow.keras as keras
import tensorflow as tf
print(tf.__version__)

2.8.0


# 日英法令対訳コーパス ダウンロード

In [6]:
!wget http://www.phontron.com/jaen-law/jaen-law.tar.gz

--2022-04-07 09:36:53--  http://www.phontron.com/jaen-law/jaen-law.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18276054 (17M) [application/gzip]
Saving to: ‘jaen-law.tar.gz.1’

jaen-law.tar.gz.1   100%[===================>]  17.43M  37.0MB/s    in 0.5s    

2022-04-07 09:36:54 (37.0 MB/s) - ‘jaen-law.tar.gz.1’ saved [18276054/18276054]



In [9]:
ls

_about.txt       jpn.txt        Predict_on_GPU.ipynb    Train_on_GPU.ipynb
checkpoints/     law-corpus.en  Predict_on_TPU.ipynb    Train_on_TPU.ipynb
image/           law-corpus.ja  preprocess_utils.py     weight_utils.py
jaen-law/        law.txt        preprocess_utils_v2.py
jaen-law.tar.gz  LICENSE        __pycache__/
jpn-law.txt      model.py       README.md


In [11]:
!tar -xzf jaen-law.tar.gz # tar.gzの解凍

# データ読み込み

In [5]:
dataset = preprocess_utils.CreateData(
    corpus_path = './jpn-law.txt',
    do_shuffle=True,
    seed_value=123,
    split_percent=0.95 # 学習データの割合
)

train_source, train_target, test_source, test_target, train_licence, test_licence = dataset.split_data()

print('**** Amount of data ****')
print('train_source： ', len(train_source))
print('train_target： ', len(train_target))
print('test_source： ', len(test_source))
print('test_target： ', len(test_target))
print('\n')
print('**** Train data example ****')
print('Source Example： ', train_source[0])
print('Target Example： ', train_target[0])
print('Licence： ', train_licence[0])
print('\n')
print('**** Test data example ****')
print('Source Example： ', test_source[0])
print('Target Example： ', test_target[0])
print('Licence： ', test_licence[0])

**** Amount of data ****
train_source：  79221
train_target：  79221
test_source：  4170
test_target：  4170


**** Train data example ****
Source Example：  My arm is broken.
Target Example：  腕を骨折しました。
Licence：  CC-BY 2.0 (France) Attribution: tatoeba.org #9524873 (DJ_Saidez) & #77115 (bunbuku)



**** Test data example ****
Source Example：  No matter how many times I've been shown, I just can't learn to tie a tie.
Target Example：  何回教えてもらってもネクタイの締め方を覚えられない。
Licence：  CC-BY 2.0 (France) Attribution: tatoeba.org #9020889 (CK) & #2432308 (tommy_san)



# 前処理

In [ ]:
BATCH_SIZE = 64 # バッチサイズ
MAX_LENGTH = 60 # シーケンスの長さ
USE_TPU = True # TPUを使うか
BUFFER_SIZE = 50000

In [ ]:
train_dataset = preprocess_utils.PreprocessData(
    mecab = MeCab.Tagger("-Ochasen"),
    source_data = train_source,
    target_data = train_target,
    max_length = MAX_LENGTH,
    batch_size = BATCH_SIZE,
    test_flag = False,
    train_dataset = None,
)

train_dataset.preprocess_data()

# バッチ作成

In [ ]:
if USE_TPU:
  tpu_grpc_url = "grpc://" + os.environ["COLAB_TPU_ADDR"]
  tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
  tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)    
  strategy = tf.distribute.TPUStrategy(tpu_cluster_resolver)

trainset = tf.data.Dataset.from_tensor_slices((train_dataset.source_vector, train_dataset.target_vector))
trainset = trainset.map(lambda source, target: (tf.cast(source, tf.int64), tf.cast(target, tf.int64))).shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

if USE_TPU:
  trainset = strategy.experimental_distribute_dataset(trainset)

INFO:tensorflow:Initializing the TPU system: grpc://10.115.145.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.115.145.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# モデル定義

In [ ]:
num_layers=4 # レイヤー数
d_model=64 # 中間層の次元数
num_heads=4 # Multi Head Attentionのヘッド数
dff=2048 # Feed Forward Networkの次元数
dropout_rate = 0.1 # ドロップアウト率

source_vocab_size = max(train_dataset.source_token.values()) + 1 # source文の語彙数
target_vocab_size = max(train_dataset.target_token.values()) + 1 # target文の語彙数

In [ ]:
# 重み初期化
def initialize_weight(checkpoint_path, optimizer, transformer, max_length, batch_size, use_tpu=False):

  if os.path.exists(checkpoint_path+'.pkl'):
    if use_tpu:
      number_of_tpu_cores = tpu_cluster_resolver.num_accelerators()['TPU']
      initialize_source, initialize_target = [[1]*max_length]*number_of_tpu_cores, [[1]*max_length]*number_of_tpu_cores
      initialize_set = tf.data.Dataset.from_tensor_slices((initialize_source, initialize_target))
      initialize_set = initialize_set.map(lambda source, target: (tf.cast(source, tf.int64), tf.cast(target, tf.int64))
          ).shuffle(buffer_size=BUFFER_SIZE).batch(batch_size).prefetch(
              buffer_size=tf.data.experimental.AUTOTUNE
          )
      initialize_set = strategy.experimental_distribute_dataset(initialize_set)

      for inp, tar in initialize_set:
        distributed_train_step(inp, tar)

    else:
      initialize_set = tf.ones([batch_size, max_length], tf.int64)
      train_step(initialize_set, initialize_set)
    
    try:
      weight_utils.load_weights_from_pickle(checkpoint_path, optimizer, transformer)
    except:
      print('Failed to load checkpoints.')

  else:
    print('No available checkpoints.')

# 学習実行

In [ ]:
with strategy.scope():
  # Transformer
  transformer = model.Transformer(num_layers, d_model, num_heads, dff,
                            source_vocab_size, target_vocab_size, 
                            pe_input=source_vocab_size, 
                            pe_target=target_vocab_size,
                            rate=dropout_rate)

  # Learning Rate
  learning_rate = model.CustomSchedule(d_model)

  # Optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                      epsilon=1e-9)

  # Loss
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

  # Loss Function
  def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

  # Metrics
  train_loss = tf.keras.metrics.Mean(name='train_loss')
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

  # Checkpoint
  #checkpoint_path = "/content/drive/My Drive/Transformer-master/checkpoints/tpu/model"
  checkpoint_path = notebook_dir+"/checkpoints/tpu/model"


  def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    
    enc_padding_mask, combined_mask, dec_padding_mask = model.create_masks(inp, tar_inp)
    
    with tf.GradientTape() as tape:
      predictions, _ = transformer(inp, tar_inp, 
                                  True, 
                                  enc_padding_mask, 
                                  combined_mask, 
                                  dec_padding_mask)
      loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    
    train_loss(loss)
    train_accuracy(tar_real, predictions)

  @tf.function
  def distributed_train_step(X, y):
    per_replica_losses = strategy.run(train_step, args=(X, y))
    return strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_losses, axis=None)

  # Initialize Weight
  initialize_weight(checkpoint_path, optimizer, transformer, MAX_LENGTH, BATCH_SIZE, use_tpu=USE_TPU)

  EPOCHS = 30
  batch = 0

  for epoch in range(EPOCHS):
    start = time.time()
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for inp, tar in trainset:
      distributed_train_step(inp, tar)
      
      if batch % 50 == 0:
        print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
            epoch + 1, batch, train_loss.result(), train_accuracy.result()))
        
      batch+=1
        
    if (epoch + 1) % 5 == 0:
      weight_utils.save_weights_as_pickle(checkpoint_path, optimizer, transformer)
      
    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                  train_loss.result(), 
                                                  train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

*** Failed to load model weights ***
['transformer_1/encoder_1/embedding_2/embeddings:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_65/kernel:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_65/bias:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_66/kernel:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_66/bias:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_67/kernel:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_67/bias:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_68/kernel:0', 'transformer_1/encoder_1/encoder_layer_4/multi_head_attention_12/dense_68/bias:0', 'dense_69/kernel:0', 'dense_69/bias:0', 'dense_70/kernel:0', 'dense_70/bias:0', 'transformer_1/encoder_1/encoder_layer_4/layer_normalization_20/gamma:0', 'transformer_1/encoder_1/encoder_layer_4/layer_normalization_20/beta:0', 'transforme